# Using different Training strategies

When training a model, training in costum strategies can be curcial, especially when re-training a model on new data.

This notebook shows how to use different training strategies for RND. 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import copy
import znrnd as znrnd
import numpy as np

import matplotlib.pyplot as plt

from neural_tangents import stax
import optax

In [ ]:
data_generator = znrnd.data.MNISTGenerator(50)
input_shape = data_generator.train_ds['inputs'][:1, ...].shape

## Define the Networks

In [ ]:
architecture = stax.serial(
    stax.Flatten(),
    stax.Dense(128),
    stax.Relu(),
    stax.Dense(128)
)

In [ ]:
target_model = znrnd.models.NTModel(
    nt_module=architecture,
    optimizer=optax.adam(learning_rate=0.02),
    input_shape=input_shape,
    batch_size=10,
)

predictor_model = znrnd.models.NTModel(
    nt_module=architecture,
    optimizer=optax.adam(learning_rate=0.02),
    input_shape=input_shape,
    batch_size=10,
)

In [ ]:
# Create a data set for recording RND
dataset = {"inputs": data_generator.train_ds["inputs"], 
           "targets": target_model(data_generator.train_ds["inputs"])}

## Define the training strategies and according recorders

Here, 3 different training strategies are presented. 
For each the train loss is recorded to show the difference between the strategies. 

1. Simple Training 
2. Partitioned Training
3. Loss aware reservoir Training

### Simple Training

In [ ]:
simple_recorder = znrnd.training_recording.JaxRecorder(
    name="simple_recorder",
    loss=True, 
    update_rate=1, 
    chunk_size=1e10 # Big Chunk-size to prevent saving the recordings.
)
simple_recorder.instantiate_recorder(
    data_set=dataset
)
simple_trainer = znrnd.training_strategies.SimpleTraining(
    model=None,
    loss_fn=znrnd.loss_functions.MeanPowerLoss(order=2),
    recorders=[simple_recorder]
)

print("SimpleTraining: \n", simple_trainer.__doc__)

### Partitioned Training

We will use that method to only train the point latest chosen data point in RND. 

In [ ]:
partitioned_recorder = znrnd.training_recording.JaxRecorder(
    name="simple_recorder",
    loss=True, 
    update_rate=1, 
    chunk_size=1e10 # Big Chunk-size to prevent saving the recordings.
)
partitioned_recorder.instantiate_recorder(
    data_set=dataset
)
partitioned_trainer = znrnd.training_strategies.PartitionedTraining(
    model=None,
    loss_fn=znrnd.loss_functions.MeanPowerLoss(order=2),
    recorders=[partitioned_recorder]
)

print("PartitionedTraining: \n", simple_trainer.__doc__)

### Loss aware reservoir Training

In [ ]:
LaR_recorder = znrnd.training_recording.JaxRecorder(
    name="simple_recorder",
    loss=True, 
    update_rate=1, 
    chunk_size=1e10 # Big Chunk-size to prevent saving the recordings.
)
LaR_recorder.instantiate_recorder(
    data_set=dataset
)
LaR_trainer = znrnd.training_strategies.LossAwareReservoir(
    model=None,
    loss_fn=znrnd.loss_functions.MeanPowerLoss(order=2),
    reservoir_size=2,
    recorders=[LaR_recorder]
)

print("LossAwareReservoir: \n", simple_trainer.__doc__)

In [ ]:
agent_dict = {
    "data_generator": data_generator,
    "target_network": target_model,
    "predictor_network": predictor_model,
    "distance_metric": znrnd.distance_metrics.OrderNDifference(order=2),
    "point_selector": znrnd.point_selection.GreedySelection(), 
}

simple_agent = znrnd.agents.RND(
        training_strategy=simple_trainer,
        **copy.deepcopy(agent_dict),
    )
partitioned_agent = znrnd.agents.RND(
        training_strategy=partitioned_trainer,
        **copy.deepcopy(agent_dict),
    )
LaR_agent = znrnd.agents.RND(
        training_strategy=LaR_trainer,
        **copy.deepcopy(agent_dict),
    )

## Execution

In [ ]:
target_size = 5
batch_size = 10
epochs = 20

_ = simple_agent.build_dataset(
    target_size=target_size, 
    epochs=epochs, 
    batch_size=batch_size,
    seed_randomly=False
)
_ = partitioned_agent.build_dataset(
    target_size=target_size, 
    epochs=[epochs], 
    batch_size=[batch_size],
    train_ds_selection=[[-1]],
    seed_randomly=False, 
)
_ = LaR_agent.build_dataset(
    target_size=target_size, 
    epochs=epochs, 
    batch_size=batch_size,
    seed_randomly=False
)

## Plot the data

In [ ]:
simple_report = simple_recorder.gather_recording()
pertitioned_report = partitioned_recorder.gather_recording()
LaR_report = LaR_recorder.gather_recording()

In [ ]:
plt.plot(simple_report.loss, '-', mfc='None', label="SimpleTraining")
plt.plot(pertitioned_report.loss, '-', mfc='None', label="PartitionedTraining")
plt.plot(LaR_report.loss, '-', mfc='None', label="LossAwareReservoir")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()